In [1]:
import keras_tuner as kt
import tensorflow as tf
from model import build_model
from utils import load_data, preprocess_data, split_data
import numpy as np

def model_builder(hp):
    n_layers = hp.Int('n_layers', 2, 5)
    n_nodes = [hp.Int(f'n_nodes_{i}', 100, 5000, step=100) for i in range(n_layers)]
    l2_lambda = [hp.Float(f'l2_lambda_{i}', 0.0001, 0.1, sampling='log') for i in range(n_layers)]
    model = build_model(n_layers, n_nodes, l2_lambda)
    return model

2025-05-16 10:17:13.745336: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load and preprocess data
X, y = load_data('../data/vacuum_data.csv')  # Or water_data.csv
X_scaled, scaler = preprocess_data(X)
X_train, y_train, X_val, y_val = split_data(X_scaled, y)

In [3]:
# Set up tuner
tuner = kt.BayesianOptimization(
    model_builder,
    objective='val_loss',
    max_trials=150,
    directory='../results/hyperparameters',
    project_name='alanine_dnn'
)

/Users/tqmthong/anaconda3/envs/biophysics-DNN/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Early stopping
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Search for best hyperparameters
tuner.search(X_train, y_train, epochs=1000, validation_data=(X_val, y_val), callbacks=[stop_early])

# Save best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

Trial 17 Complete [00h 15m 50s]
val_loss: 0.6001011729240417

Best val_loss So Far: 0.5977813005447388
Total elapsed time: 05h 36m 01s

Search: Running Trial #18

Value             |Best Value So Far |Hyperparameter
4                 |3                 |n_layers
4900              |2800              |n_nodes_0
4500              |4700              |n_nodes_1
0.005177          |0.00020566        |l2_lambda_0
0.072538          |0.00021128        |l2_lambda_1
4300              |4000              |n_nodes_2
4600              |4800              |n_nodes_3
0.00011424        |0.0032808         |l2_lambda_2
0.001157          |0.0088273         |l2_lambda_3
4500              |800               |n_nodes_4
0.00025024        |0.00013933        |l2_lambda_4

Epoch 1/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 177s 699ms/step - loss: 96.8852 - val_loss: 3.4265
Epoch 2/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 176s 701ms/step - loss: 2.5513 - val_loss: 0.9980
Epoch 3/1000
250/250 ━━━━━━━━━━━━━━━━━━━━ 165s 661ms/step - l

In [ ]:
with open('../results/hyperparameters/best_hps.txt', 'w') as f:
    f.write(f"n_layers: {best_hps.get('n_layers')}\n")
    for i in range(best_hps.get('n_layers')):
        f.write(f"n_nodes_{i}: {best_hps.get(f'n_nodes_{i}')}\n")
        f.write(f"l2_lambda_{i}: {best_hps.get(f'l2_lambda_{i}')}\n")